In [33]:
import numpy as np
from pgmpy.models import MarkovModel
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.inference import BeliefPropagation

# a) Crearea imaginii originale
original_image = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
])

# b) Adăugarea zgomotului
np.random.seed(42)
noisy_image = original_image.copy()
rows, cols = original_image.shape
indices = np.random.choice(rows * cols, size=3, replace=False)
for idx in indices:
    row = idx // cols
    col = idx % cols
    noisy_image[row, col] = 1 - noisy_image[row, col]

# c) Definirea MRF cu pgmpy
model = MarkovModel()

# Adăugarea nodurilor și muchiilor
nodes = [(i, j) for i in range(5) for j in range(5)]
model.add_nodes_from(nodes)

edges = []
for i in range(5):
    for j in range(5):
        if i < 4:
            edges.append(((i, j), (i+1, j)))
        if j < 4:
            edges.append(((i, j), (i, j+1)))
model.add_edges_from(edges)

# Factorii pentru noduri (observații)
lambda_val = 10.0  # Ajustat pentru a da mai multă importanță observațiilor
for i in range(5):
    for j in range(5):
        y = noisy_image[i, j]
        potential = [np.exp(-lambda_val * (0 - y)**2), np.exp(-lambda_val * (1 - y)**2)]
        factor = DiscreteFactor([(i, j)], [2], potential)
        model.add_factors(factor)

# Factorii pentru perechi de vecini
for edge in edges:
    (i1, j1), (i2, j2) = edge
    potential = [1.0, np.exp(-1), np.exp(-1), 1.0]  # (0,0), (0,1), (1,0), (1,1)
    factor = DiscreteFactor([(i1, j1), (i2, j2)], [2, 2], potential)
    model.add_factors(factor)

model.check_model()

# d) Estimarea MAP cu Belief Propagation
bp = BeliefPropagation(model)
map_result = bp.map_query()

# Convertim rezultatul în matrice
estimated_image = np.zeros((5, 5), dtype=int)
for (i, j), val in map_result.items():
    estimated_image[i, j] = val

print("Imaginea estimată:")
print(estimated_image)

Imaginea estimată:
[[1 0 0 0 0]
 [0 1 1 0 0]
 [0 1 0 1 0]
 [0 0 1 1 0]
 [0 0 0 0 0]]
